In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
base_path = Path().resolve().parent
%cd {base_path}

/home/optima/mhaderer/AILS-MICCAI-UWF4DR-Challenge


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# setup
#!apt-get update
#!pip install python-dotenv
#!pip install loguru
#!pip install gdown
#!pip install typer
#!pip install imbalanced-learn

In [4]:
# load data and unzip data

#!python tools/download_data_and_chkpts.py

In [5]:
# imports

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
import wandb

from sklearn.metrics import roc_auc_score, average_precision_score

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import time

# data
from ails_miccai_uwf4dr_challenge.dataset_strategy import CustomDataset, DatasetStrategy, CombinedDatasetStrategy, \
    Task1Strategy, Task2Strategy, Task3Strategy, TrainValSplitStrategy, RandomOverSamplingStrategy, DatasetBuilder

# augmentation
from ails_miccai_uwf4dr_challenge.preprocess_augmentations import ResidualGaussBlur, MultiplyMask

# trainer
from ails_miccai_uwf4dr_challenge.models.trainer import DefaultMetricsEvaluationStrategy, Metric, MetricCalculatedHook, \
    NumBatches, Trainer, EpochTrainingStrategy, EpochValidationStrategy, DefaultEpochTrainingStrategy, \
    DefaultBatchTrainingStrategy, TrainingContext, PersistBestModelOnEpochEndHook
from ails_miccai_uwf4dr_challenge.models.metrics import sensitivity_score, specificity_score
from ails_miccai_uwf4dr_challenge.config import Config

# models
from ails_miccai_uwf4dr_challenge.models.architectures.task1_automorph_plain import AutoMorphModel
from ails_miccai_uwf4dr_challenge.models.architectures.task1_efficientnet_plain import Task1EfficientNetB4
from ails_miccai_uwf4dr_challenge.models.architectures.task2_efficientnetb0_plain import Task2EfficientNetB0 
from ails_miccai_uwf4dr_challenge.models.architectures.task1_convnext import Task1ConvNeXt 

from ails_miccai_uwf4dr_challenge.config import WANDB_API_KEY, PROJ_ROOT

wandb.login(key=WANDB_API_KEY)

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.11 (you have 1.4.7). Upgrade using: pip install --upgrade albumentations
wandb: Currently logged in as: mmarinschek (miccai-challenge-2024). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/optima/mhaderer/.netrc


True

In [6]:
# select device for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))

Device: cuda


## Train some model

In [12]:
def train_model(cfg=None):

    WANDB_HTTP_TIMEOUT=300
    WANDB_INIT_TIMEOUT =600
    WANDB_DEBUG=True

    wandb.init(project="task2", config=cfg)
    cfg = wandb.config

    transforms_train = A.Compose([
        A.Resize(800, 1016, p=1),
        MultiplyMask(p=1),
        ResidualGaussBlur(p=cfg.p_gaussblur),
        A.Equalize(p=cfg.p_equalize),
        A.CLAHE(clip_limit=5., p=cfg.p_clahe),
        A.HorizontalFlip(p=cfg.p_horizontalflip),
        A.Affine(rotate=cfg.rotation, rotate_method='ellipse', p=cfg.p_affine),
        A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
        ToTensorV2(p=1)
    ])

    transforms_val = A.Compose([
            A.Resize(800, 1016, p=1),
            MultiplyMask(p=1),
            A.Normalize(mean=[0.406, 0.485, 0.456], std=[0.225, 0.229, 0.224], p=1),
            ToTensorV2(p=1)
        ])

    dataset_strategy = CombinedDatasetStrategy()
    task_strategy = Task2Strategy()
    
    split_strategy = TrainValSplitStrategy(split_ratio=0.8)
    resampling_strategy = RandomOverSamplingStrategy()

    builder = DatasetBuilder(dataset_strategy, task_strategy, split_strategy, resampling_strategy)
    train_data, val_data = builder.build()

    train_dataset = CustomDataset(train_data, transform=transforms_train)
    val_dataset = CustomDataset(val_data, transform=transforms_val)

    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=cfg.batch_size, shuffle=False)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu" if torch.backends.mps.is_available() else "mps")
    print(f"Using device: {device}")

    assert cfg.model is not None, "Model type must be specified in the config"

    if cfg.model == "Task2EfficientNetB0":
        model = Task2EfficientNetB0(num_classes=1)
    elif cfg.model == "AutoMorphModel":
        model = AutoMorphModel(enc_frozen=True)
    #elif cfg.model == "Task1ConvNeXt":
    #    model = Task1ConvNeXt()

    model.to(device)

    metrics = [
        Metric('auroc', roc_auc_score),
        Metric('auprc', average_precision_score),
        Metric('accuracy', lambda y_true, y_pred: (y_pred.round() == y_true).mean()),
        Metric('sensitivity', sensitivity_score),
        Metric('specificity', specificity_score)
    ]

    class WandbLoggingHook(MetricCalculatedHook):
        def on_metric_calculated(self, training_context: TrainingContext, metric: Metric, result, last_metric_for_epoch: bool):
            import wandb
            wandb.log(data={metric.name: result}, commit=last_metric_for_epoch)

    metrics_eval_strategy = DefaultMetricsEvaluationStrategy(metrics).register_metric_calculated_hook(WandbLoggingHook())

    def combined_losses(pred, target):
        bce = F.binary_cross_entropy_with_logits(pred, target) * cfg.loss_weight
        smooth_l1 = F.smooth_l1_loss(pred, target) * (1 - cfg.loss_weight)
        return bce + smooth_l1

    criterion = combined_losses
    optimizer = optim.AdamW(model.parameters(), lr=cfg.lr)
    lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=cfg.lr_schedule_factor, patience=cfg.lr_schedule_patience, verbose=True)

    trainer = Trainer(model, train_loader, val_loader, criterion, optimizer, lr_scheduler, device, 
                        metrics_eval_strategy=metrics_eval_strategy)

    # build a file name for the model weights containing current timestamp and the model class
    training_timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
    persist_model_hook = PersistBestModelOnEpochEndHook(f"models/best_model_{training_timestamp}.pth")
    trainer.add_epoch_end_hook(persist_model_hook) # TODO uncomment this line to save the best model

    #print("First train 2 epochs 2 batches to check if everything works - you can comment these two lines after the code has stabilized...")
    #trainer.train(num_epochs=2, num_batches=NumBatches.TWO_FOR_INITIAL_TESTING)
    
    print("Now train train train")
    trainer.train(num_epochs=cfg.epochs)
    #wandb.finish()
    print("Finished training")

In [13]:
cfg = Config(
    batch_size=16,
    epochs=25,
    lr=0.001,
    lr_schedule_factor=0.1,
    lr_schedule_patience=5,
    p_gaussblur=0,
    p_equalize=0,
    p_clahe=0.3,
    p_horizontalflip=0.3,
    rotation=15,
    p_affine=0.3,
    loss_weight=0.5,
    model="Task2EfficientNetB0",
)

In [14]:
train_model(cfg)


Original class distribution:
Class 0: 149
Class 1: 250
dr
1    0.626566
0    0.373434
Name: proportion, dtype: Float64

Resampled class distribution:
dr
1    0.5
0    0.5
Name: proportion, dtype: Float64
Using device: cuda
Loaded pretrained weights for efficientnet-b0


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Now train train train


Epoch 1/15 - Avg val Loss: 0.442960: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.48s/it]


New best model found at epoch 1 with validation loss: 0.4430. Model saved to models/best_model_2024-07-23_16-29-23.pth


...

New best model found at epoch 12 with validation loss: 0.2950. Model saved to models/best_model_2024-07-23_16-29-23.pth


Epoch 13/15 - Avg val Loss: 0.293087: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:09<00:00,  1.43s/it]


New best model found at epoch 13 with validation loss: 0.2931. Model saved to models/best_model_2024-07-23_16-29-23.pth


Epoch 15/15 - Avg val Loss: 0.335347: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:10<00:00,  1.45s/it]

Finished training


## SWEEP

In [15]:

# Define the sweep configuration
sweep_config = {
    "method": "random",  # or "grid", or "bayes"
    "parameters": {
        "model": {
            "values": ["Task2EfficientNetB0"]
        },
        "lr": {
            "values": [1e-3]
        },
        "epochs": {
            "values": [20]
        },
        "batch_size": {
            "values": [16, 32]
        },
        "p_gaussblur": {
            "values": [0, 0.3]
        },
        "p_equalize": {
            "values": [0, 0.3]
        },
        "p_clahe": {
            "values": [0, 0.3]
        },
        "p_horizontalflip": {
            "values": [0, 0.3]
        },
        "rotation": {
            "values": [15, 30]
        },
        "p_affine": {
            "values": [0, 0.3]
        },
        "loss_weight": {
            "values": [0.5, 0.7]
        },
        "lr_schedule_factor": {
            "values": [0.1]
        },
        "lr_schedule_patience": {
            "values": [5]
        }
    }
}

# Initialize the sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="task2")

# Start the sweep
wandb.agent(sweep_id, function=train_model)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: wku4858f
Sweep URL: https://wandb.ai/miccai-challenge-2024/task2/sweeps/wku4858f
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>
<IPython.core.display.HTML object>


wandb: Agent Starting Run: msrumx0e with config:
wandb: 	batch_size: 32
wandb: 	epochs: 20
wandb: 	loss_weight: 0.7
wandb: 	lr: 0.001
wandb: 	lr_schedule_factor: 0.1
wandb: 	lr_schedule_patience: 5
wandb: 	model: Task2EfficientNetB0
wandb: 	p_affine: 0
wandb: 	p_clahe: 0
wandb: 	p_equalize: 0
wandb: 	p_gaussblur: 0.3
wandb: 	p_horizontalflip: 0
wandb: 	rotation: 30
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


/opt/conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "



Original class distribution:
Class 0: 149
Class 1: 250
dr
1    0.626566
0    0.373434
Name: proportion, dtype: Float64

Resampled class distribution:
dr
1    0.5
0    0.5
Name: proportion, dtype: Float64
Using device: cuda
Loaded pretrained weights for efficientnet-b0
Now train train train


...

New best model found at epoch 5 with validation loss: 0.4856. Model saved to models/best_model_2024-07-23_19-26-44.pth


Epoch 6/20 - Avg train Loss: 0.348865:  85%|███████████████████████████████████████████████████████████████████████████████████████████████████████████▍                   | 11/13 [00:33<00:06,  3.19s/it]wandb: Ctrl + C detected. Stopping sweep.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fc115400220>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fc115425ab0, execution_count=15 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fc115425ae0, raw_cell="
# Define the sweep configuration
sweep_config = {.." store_history=True silent=False shell_futures=True cell_id=f4f9641f-67ff-4619-a08a-c6f25da9291c> result=None>,),kwargs {}:


BrokenPipeError: [Errno 32] Broken pipe

Epoch 6/20 - Starting validation...:   0%|                                                                                                                                           | 0/4 [00:00<?, ?it/s]